# Constraint Methods Examples

This notebook solves the three problems using penalty and barrier methods via `optymus.Optimizer`.
Barrier runs require a strictly feasible starting point (all `g(x) < 0`).

In [ ]:
import jax.numpy as jnp

from optymus import Optimizer


def print_result(label, result, g_cons=None, h_cons=None):
    xopt = result["xopt"]
    fmin = result["fmin"]
    print(label)
    print(f"  xopt = {xopt}")
    print(f"  fmin = {fmin}")
    if g_cons:
        vals = [g(xopt) for g in g_cons]
        print(f"  g(x) = {vals}")
    if h_cons:
        vals = [h(xopt) for h in h_cons]
        print(f"  h(x) = {vals}")
    if result.get("warnings"):
        print(f"  warnings = {result["warnings"]}")


## Problem 1

Minimize: `f(x1, x2) = (x1 - 2)^2 + (x2 - 2)^2`
Subject to: `x1 + x2 + 3 <= 0`

Parameters:
- Penalty: `r_p0 = 0.1`, `beta = 10`, `x0 = {-5, -2}`
- Barrier: `r_b0 = 10`, `beta = 0.1`, `x0 = {-5, -2}`

In [ ]:
def f1(x):
    return (x[0] - 2.0) ** 2 + (x[1] - 2.0) ** 2

def g1(x):
    return x[0] + x[1] + 3.0

x0_penalty = jnp.array([-5.0, -2.0])
x0_barrier = jnp.array([-5.0, -2.0])

opt_penalty = Optimizer(
    f_obj=f1,
    g_cons=[g1],
    x0=x0_penalty,
    method="adam",
    constraint_method="penalty",
    penalty_r0=0.1,
    penalty_factor=10.0,
    max_outer_iter=6,
    max_iter=200,
    learning_rate=0.05,
    verbose=False,
)
print_result("Problem 1 - Penalty", opt_penalty.get_results(), g_cons=[g1])

opt_barrier = Optimizer(
    f_obj=f1,
    g_cons=[g1],
    x0=x0_barrier,
    method="adam",
    constraint_method="barrier",
    barrier_r0=10.0,
    barrier_factor=0.1,
    penalty_r0=0.1,
    penalty_factor=10.0,
    max_outer_iter=6,
    max_iter=200,
    learning_rate=0.05,
    verbose=False,
)
print_result("Problem 1 - Barrier", opt_barrier.get_results(), g_cons=[g1])


## Problem 2

Minimize: `f(x1, x2) = (x1 - 2)^4 + (x1 - 2 x2)^2`
Subject to: `x1^2 - x2 <= 0`

Parameters:
- Penalty: `r_p0 = 1`, `beta = 10`, `x0 = {3, 2}`
- Barrier: `r_b0 = 10`, `beta = 0.1`, `x0 = {0, 1}`

In [ ]:
def f2(x):
    return (x[0] - 2.0) ** 4 + (x[0] - 2.0 * x[1]) ** 2

def g2(x):
    return x[0] ** 2 - x[1]

x0_penalty = jnp.array([3.0, 2.0])
x0_barrier = jnp.array([0.0, 1.0])

opt_penalty = Optimizer(
    f_obj=f2,
    g_cons=[g2],
    x0=x0_penalty,
    method="adam",
    constraint_method="penalty",
    penalty_r0=1.0,
    penalty_factor=10.0,
    max_outer_iter=6,
    max_iter=200,
    learning_rate=0.05,
    verbose=False,
)
print_result("Problem 2 - Penalty", opt_penalty.get_results(), g_cons=[g2])

opt_barrier = Optimizer(
    f_obj=f2,
    g_cons=[g2],
    x0=x0_barrier,
    method="adam",
    constraint_method="barrier",
    barrier_r0=10.0,
    barrier_factor=0.1,
    penalty_r0=1.0,
    penalty_factor=10.0,
    max_outer_iter=6,
    max_iter=200,
    learning_rate=0.05,
    verbose=False,
)
print_result("Problem 2 - Barrier", opt_barrier.get_results(), g_cons=[g2])


## Problem 3

Two-bar truss weight minimization. Variables are the mean diameter `d` and height `H`.
Constraints: axial stress must not exceed yield and Euler buckling stress.

Parameters:
- Penalty: `r_p0 = 1e-7`, `beta = 10`, `x0 = {1, 15}`
- Barrier: `r_b0 = 1e7`, `beta = 0.1`, `x0 = {4, 25}`

In [ ]:
rho = 0.3
B = 30.0
P = 33e3
t = 0.1
E = 3e7
sigma_y = 1e5

def truss_geometry(x):
    d = x[0]
    H = x[1]
    L = jnp.sqrt(B**2 + H**2)
    return d, H, L

def area(d):
    return jnp.pi * d * t

def inertia(d):
    return jnp.pi * d**3 * t / 8.0

def axial_force(H, L):
    return P * L / H

def truss_stresses(x):
    d, H, L = truss_geometry(x)
    A = area(d)
    I = inertia(d)
    N = axial_force(H, L)
    axial = N / A
    euler = (jnp.pi**2 * E * I) / (L**2 * A)
    return d, L, axial, euler

def f3(x):
    d, L, _, _ = truss_stresses(x)
    return 2.0 * rho * area(d) * L

def g3_constraints(x):
    _, _, axial, euler = truss_stresses(x)
    return jnp.array([axial - sigma_y, axial - euler])

x0_penalty = jnp.array([1.0, 15.0])
x0_barrier = jnp.array([4.0, 25.0])
bounds = (jnp.array([0.1, 0.1]), jnp.array([50.0, 50.0]))

opt_penalty = Optimizer(
    f_obj=f3,
    g_cons=[g3_constraints],
    x0=x0_penalty,
    method="adam",
    constraint_method="penalty",
    penalty_r0=1e-7,
    penalty_factor=10.0,
    max_outer_iter=8,
    max_iter=300,
    learning_rate=0.05,
    bounds=bounds,
    verbose=False,
)
print_result("Problem 3 - Penalty", opt_penalty.get_results(), g_cons=[g3_constraints])

opt_barrier = Optimizer(
    f_obj=f3,
    g_cons=[g3_constraints],
    x0=x0_barrier,
    method="adam",
    constraint_method="barrier",
    barrier_r0=1e7,
    barrier_factor=0.1,
    penalty_r0=1e-7,
    penalty_factor=10.0,
    max_outer_iter=8,
    max_iter=300,
    learning_rate=0.05,
    bounds=bounds,
    verbose=False,
)
print_result("Problem 3 - Barrier", opt_barrier.get_results(), g_cons=[g3_constraints])
